In [25]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd "/content/drive/MyDrive/Colab Notebooks/SemEval2023/Evaluation"

In [26]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install colorama
!pip install wikipedia-api
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [27]:
import sys
import os
sys.path.append('../')
import pandas as pd
import torch 
import numpy as np
from tqdm import tqdm
import random
from datasets import Dataset
import nltk
nltk.download('punkt')
os.environ["WANDB_DISABLED"] = "true"
from helper import prepare_data


[nltk_data] Downloading package punkt to /home/hossain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

### Seed all

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [29]:
LANG = 'fa' # use None for all lang
SET = 'LM' # 'LM' or None
EVAL_SET = 'test'

In [30]:
if LANG=='en' and SET=='LM':
    !python -m spacy download en_core_web_lg

## Read Data

In [31]:
filename = f'../Dataset/{LANG}-{EVAL_SET}.conll'
data = prepare_data(filename)

In [32]:
data.head()

,ID,lang,sent,labels
0,c03012f9-9724-41ff-a5d7-7f7962867204,None,قراردادی که دولت بریتانیا سعی در بستن آ...,
1,e37693f1-2c16-490c-82e4-95d30ff133e8,None,اِپُک تایمز ، یک شرکت رسانه ای خبری در...,
2,3e47c352-2834-4dc3-aa42-de202bf3a1e4,None,این فرودگاه در کشور ایالات متحده آمریکا ...,
3,0884cdd1-3c28-48b3-9493-8bfaad6de6ac,None,فینیکس جمعیت ۱ ٬ ۴۴۵ ٬ ۶۳۲,
4,2e91a7ff-8581-46ab-a8a5-e2829d03b110,None,هنزیک ( honzik ) ، حشره ای است که در ...,


## Augment Info

In [33]:
from InformationExtraction import InformationExtractionPipeline
infoPipeline = InformationExtractionPipeline(SET,
                                        max_sen = 2, lang = LANG, 
                                        loadJson = True, jsonPath=f'./Wiki/{LANG}-wiki.json',
                                            saveJson=True, saveJsonpath = f'./Wiki/{LANG}-test-wiki.json')

In [34]:
augmented = infoPipeline(data[['sent','labels']].values.tolist())


  0%|                                   | 542/219168 [03:50<68:35:11,  1.13s/it]

HTTPSConnectionPool(host='fa.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&prop=info&titles=%DA%AF%D9%88%D8%B1%D9%86%DA%AF%D8%A7_%D8%AA%D9%88%D9%BE%D9%88%D9%86%DB%8C%D8%AA%D8%B3%D8%A7&inprop=protection%7Ctalkid%7Cwatched%7Cwatchers%7Cvisitingwatchers%7Cnotificationtimestamp%7Csubjectid%7Curl%7Creadable%7Cpreload%7Cdisplaytitle&format=json&redirects=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe1b0712c20>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


  2%|▊                                 | 5349/219168 [35:36<34:56:11,  1.70it/s]

HTTPSConnectionPool(host='fa.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


 13%|████                           | 28397/219168 [2:42:56<22:16:52,  2.38it/s]

...

 65%|██████████████████▉          | 142711/219168 [14:07:25<10:45:56,  1.97it/s]

HTTPSConnectionPool(host='fa.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&prop=info&titles=%D8%AC%D9%88%D9%86%D8%A7%D8%B3_%D8%B3%D8%A7%D9%88%DB%8C%D9%85%D8%A8%DB%8C&inprop=protection%7Ctalkid%7Cwatched%7Cwatchers%7Cvisitingwatchers%7Cnotificationtimestamp%7Csubjectid%7Curl%7Creadable%7Cpreload%7Cdisplaytitle&format=json&redirects=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe1b0b84e20>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


 97%|██████████████████████████████▉ | 211896/219168 [17:57:44<20:29,  5.91it/s]

HTTPSConnectionPool(host='fa.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&prop=info&titles=%D8%A7%D8%B3_._%D8%A7%D9%85_.&inprop=protection%7Ctalkid%7Cwatched%7Cwatchers%7Cvisitingwatchers%7Cnotificationtimestamp%7Csubjectid%7Curl%7Creadable%7Cpreload%7Cdisplaytitle&format=json&redirects=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe1b0f8bbb0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))


100%|████████████████████████████████| 219168/219168 [18:14:40<00:00,  3.34it/s]


In [35]:
data['augmented_sen'] = augmented
temp = data[data['sent']!=data['augmented_sen']]
info_percent = temp.shape[0]/data.shape[0]
print(f"Info Percentage: {info_percent*100:.2f}%")

Info Percentage: 52.17%


In [36]:
dir = f'./Augmented-Dataset'
if not os.path.exists(dir):
    os.mkdir(dir)
data.to_csv(f'{dir}/{LANG}-test.csv',index=False)

In [38]:
from datasets import load_dataset, Dataset

data = Dataset.from_pandas(data)

In [39]:
data

Dataset({
    features: ['ID', 'lang', 'sent', 'labels', 'augmented_sen'],
    num_rows: 219168
})